[Reference](https://medium.com/python-pandemonium/top-design-patterns-in-python-9778843d5451)

# Iterator

In [2]:
from __future__ import annotations
from collections.abc import Iterable, Iterator
from typing import Any, List


class AlphabeticalOrderIterator(Iterator):
    _position: int = None
    _reverse: bool = False

    def __init__(self, collection: WordsCollection, 
                 reverse: bool = False):
        self._collection = collection
        self._reverse = reverse
        self._position = -1 if reverse else 0

    def __next__(self):
        try:
            value = self._collection[self._position]
            self._position += -1 if self._reverse else 1
        except IndexError:
            raise StopIteration()
        return value


class WordsCollection(Iterable):
    def __init__(self, collection: List[Any] = []):
        self._collection = collection

    def __iter__(self) -> AlphabeticalOrderIterator:
        return AlphabeticalOrderIterator(self._collection)

    def get_reverse_iterator(self) -> AlphabeticalOrderIterator:
        return AlphabeticalOrderIterator(self._collection, True)

    def add_item(self, item: Any):
        self._collection.append(item)


if __name__ == "__main__":
    collection = WordsCollection()
    collection.add_item("First")
    collection.add_item("Second")
    collection.add_item("Third")

    print("Straight traversal:")
    print("\n".join(collection))

    print("Reverse traversal:")
    print("\n".join(collection.get_reverse_iterator()))

# State

In [3]:
from __future__ import annotations
from abc import ABC, abstractmethod

class Context(ABC):
    _state = None    
    def __init__(self, state: State):
        self.transition_to(state)

    def transition_to(self, state: State):
        print(f"Context: Transition to {type(state).__name__}")
        self._state = state
        self._state.context = self    
    
    def request1(self):
        self._state.handle1()    
    
    def request2(self):
        self._state.handle2()

class State(ABC):
    @property
    def context(self) -> Context:
        return self._context

    @context.setter
    def context(self, context: Context):
        self._context = context 

    @abstractmethod
    def handle1(self):
        pass    
    
    @abstractmethod
    def handle2(self):
        pass

class ConcreteStateA(State):
    def handle1(self):
        print("ConcreteStateA handles request1.")
        print("ConcreteStateA wants to change the state of the context.")
        self.context.transition_to(ConcreteStateB())

    def handle2(self):
        print("ConcreteStateA handles request2.")

class ConcreteStateB(State):
    def handle1(self):
        print("ConcreteStateB handles request1.")   

    def handle2(self):
        print("ConcreteStateB handles request2.")
        print("ConcreteStateB wants to change the state of the context.")
        self.context.transition_to(ConcreteStateA())
        
if __name__ == "__main__":
    context = Context(ConcreteStateA())
    context.request1()
    context.request2()

# Observer

In [5]:
from __future__ import annotations
from abc import ABC, abstractmethod
from random import randrange
from typing import List

class Subject(ABC):
    @abstractmethod
    def attach(self, observer: Observer):
        pass    
    
    @abstractmethod
    def detach(self, observer: Observer):
        pass    
    
    @abstractmethod
    def notify(self):
        pass
class ConcreteSubject(Subject):
    _state: int = None
    _observers: List[Observer] = []
   
    def attach(self, observer: Observer):
        print("Subject: Attached an observer.")
        self._observers.append(observer)

    def detach(self, observer: Observer):
        self._observers.remove(observer)

    def notify(self):
        print("Subject: Notifying observers...")
        for observer in self._observers:
            observer.update(self)    
            
    def some_business_logic(self):
        print("Subject: I'm doing something important.")
        self._state = randrange(0, 10)
        print(f"Subject: My state has just changed to: {self._state}")
        self.notify()
class Observer(ABC):
    @abstractmethod
    def update(self, subject: Subject):       
        pass

class ConcreteObserverA(Observer):
    def update(self, subject: Subject):
        if subject._state < 3:
            print("ConcreteObserverA: Reacted to the event")

class ConcreteObserverB(Observer):
    def update(self, subject: Subject):
        if subject._state == 0 or subject._state >= 2:
            print("ConcreteObserverB: Reacted to the event")

if __name__ == "__main__":    
    subject = ConcreteSubject()    
    
    observer_a = ConcreteObserverA()
    subject.attach(observer_a)    
    
    observer_b = ConcreteObserverB()
    subject.attach(observer_b)    
    
    subject.some_business_logic()
    subject.some_business_logic()    
    
    subject.detach(observer_a)    
    
    subject.some_business_logic()

# Facade

In [6]:
class Addition:
    def __init__(self, field1: int, field2: int):
        self.field1 = field1
        self.field2 = field2    
        
    def get_result(self):
        return self.field1 + self.field2
class Multiplication:
    def __init__(self, field1: int, field2: int):
        self.field1 = field1
        self.field2 = field2    
        
    def get_result(self):
        return self.field1 * self.field2
class Subtraction:
    def __init__(self, field1: int, field2: int):
        self.field1 = field1
        self.field2 = field2    
        
    def get_result(self):
        return self.field1 - self.field2
class Facade:
    @staticmethod
    def make_addition(*args) -> Addition:
        return Addition(*args)    
    @staticmethod
    def make_multiplication(*args) -> Multiplication:
        return Multiplication(*args)    
    @staticmethod
    def make_subtraction(*args) -> Subtraction:
        return Subtraction(*args)

if __name__ == "__main__":
    addition_obj = Facade.make_addition(5, 5)
    multiplication_obj = Facade.make_multiplication(5, 2)
    subtraction_obj = Facade.make_subtraction(15, 5)    
    
    print(addition_obj.get_result())
    print(multiplication_obj.get_result())
    print(subtraction_obj.get_result())

# Decorator

In [7]:
class my_decorator:
    def __init__(self, func):
        print("inside my_decorator.__init__()")
        func() # Prove that function definition has completed    
        
    def __call__(self):
        print("inside my_decorator.__call__()")
@my_decorator
def my_function():
    print("inside my_function()")
if __name__ == "__main__":    
    my_function()

# Adapter

In [8]:
class Target:
    def request(self):
        return "Target: The default target's behavior."

class Adaptee:
    def specific_request(self):
        return ".eetpadA eht fo roivaheb laicepS"

class Adapter(Target, Adaptee):
    def request(self):
        return f"Adapter: (TRANSLATED) {self.specific_request()[::-1]}"

def client_code(target: "Target"):
    print(target.request())

if __name__ == "__main__":
    print("Client: I can work just fine with the Target objects:")    target = Target()
    client_code(target)    
    
    adaptee = Adaptee()
    
    print("Client: The Adaptee class has a weird interface. "
          "See, I don't understand it:")
    print(f"Adaptee: {adaptee.specific_request()}")    
    
    print("Client: But I can work with it via the Adapter:")
    
    adapter = Adapter()
    client_code(adapter)

# Singleton

In [9]:
class Singleton:
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

if __name__ == "__main__":
    s = Singleton()
    print("Object created:", s)    
    
    s1 = Singleton()
    print("Object created:", s1)